<a href="https://colab.research.google.com/github/Nikita785/BLCMods/blob/master/day2_semifinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df_fd = pd.read_csv('/content/financial_data.csv')
df_pr = pd.read_csv('/content/prolongations.csv')

In [6]:
#  Шаг первый: очистка данных
#    - Счел нужным удалить Причины дубля из df_fd для удобства
#    - Преобразовать месяцы в df_fd в числовой тип и заполнить NaN нулями для реализации вычислений

df_fd = df_fd.drop('Причина дубля', axis=1)
cols_to_convert = ['Ноябрь 2022',
                   'Декабрь 2022', 'Январь 2023', 'Февраль 2023',
                   'Март 2023', 'Апрель 2023', 'Май 2023',
                   'Июнь 2023', 'Июль 2023', 'Август 2023',
                   'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023',
                   'Декабрь 2023', 'Январь 2024', 'Февраль 2024']

for col in cols_to_convert:
  df_fd[col] = df_fd[col].astype(str).str.replace('\xa0', '').str.replace(',', '.')
  df_fd[col] = pd.to_numeric(df_fd[col], errors='coerce').fillna(0)

In [7]:
# Шаг второй: объединение таблиц для произведения расчётов

df_merge = pd.merge(df_fd, df_pr, on='id', how='left')

In [19]:
#Шаг третий: сам расчёт коэфициентов пролонгации

#Итерация по месяцам
# Словарь соответствия для русских названий месяцев и их номеров
month_dict = {
    'Январь 2023': 1, 'Февраль 2023': 2, 'Март 2023': 3, 'Апрель 2023': 4,
    'Май 2023': 5, 'Июнь 2023': 6, 'Июль 2023': 7, 'Август 2023': 8,
    'Сентябрь 2023': 9, 'Октябрь 2023': 10, 'Ноябрь 2023': 11, 'Декабрь 2023': 12,
    'Ноябрь 2022': 11, 'Декабрь 2022': 12  # Добавьте 2022 год
}



import pandas as pd

# Функция для расчета коэффициентов пролонгации за заданный месяц
def calculate_coef(df_merge, month):  # Исправлено: df_merge -> df_merged
    month_year = month.split()
    month_num = month_dict[month]
    year_num = int(month_year[1])

    # Определение предыдущего и позапрошлого месяцев
    prev_month_num = month_num - 1
    prev_year_num = year_num
    if prev_month_num == 0:
        prev_month_num = 12
        prev_year_num -= 1
    prev_month = pd.to_datetime(prev_month_num, format='%m').strftime('%B') + ' ' + str(prev_year_num)

    prev2_month_num = prev_month_num - 1
    prev2_year_num = prev_year_num #Исправлено: prev_year -> prev_year_num
    if prev2_month_num == 0:
        prev2_month_num = 12
        prev2_year_num -= 1
    if prev2_month_num == -1:
        prev2_month_num = 11
        prev2_year_num -= 1
    prev2_month = pd.to_datetime(prev2_month_num, format='%m').strftime('%B') + ' ' + str(prev2_year_num)

    # Пролонгация в первый месяц
    # Проекты, которые завершились в предыдущем месяце
    completed_prev_month = df_merge[df_merge['month'] == prev_month]
    # Сумма завершившихся проектов
    prev_month_sum = completed_prev_month[prev_month].sum() #Исправлено: prev_monrh_sum -> prev_month_sum
    # Фильтр на пролонгированные проекты
    cpm_filtr_long = completed_prev_month[completed_prev_month[month] > 0]
    # Сумма пролонгированных проектов
    cpm_filtr_sum = cpm_filtr_long[month].sum()
    # Вычисление коэффициента пролонгированности за первый месяц (отношение к пролонгированным проектам ко всем)
    cpm_ratio = (cpm_filtr_sum / prev_month_sum) if prev_month_sum else 0

    # Пролонгация во второй месяц
    # Фильтрация проектов, завершившихся за 2 месяца до текущего
    completed_prev2_month = df_merge[df_merge['month'] == prev2_month]
    # Фильтрация проектов, завершившихся за 2 месяца, которые не были пролонгированы в прошлом
    cp2m_filtr_notlong = completed_prev2_month[completed_prev2_month[prev_month] == 0]
    # Сумма проектов, завершившихся за 2 месяца до текущего и непролонгированные в прошлом
    cp2m_filtr_sum = cp2m_filtr_notlong[prev2_month].sum()
    # Фильтрация пролонгированных проектов
    cp2m_filtr_long = cp2m_filtr_notlong[cp2m_filtr_notlong[month] > 0]
    # Сумма пролонгированных проектов
    cp2m_filtr_long_sum = cp2m_filtr_long[month].sum()
    # Вычисление коэффициента пролонгированности за 2 месяц (то же самое)
    cp2m_ratio = (cp2m_filtr_long_sum / cp2m_filtr_sum) if cp2m_filtr_sum else 0

    return cpm_ratio, cp2m_ratio

In [20]:
#Шаг четвёртый: агрегация данных

months = ['Январь 2023', 'Февраль 2023',
          'Март 2023', 'Апрель 2023', 'Май 2023',
          'Июнь 2023', 'Июль 2023', 'Август 2023',
          'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023',
          'Декабрь 2023']

results = []

for month in months:
  coef1, coef2 = calculate_coef(df_merge, month)
  results.append({'Месяц':month, 'Коэфициент 1': coef1, 'Коэфициент 2': coef2})
df_results = pd.DataFrame(results)

#Агрегация по году, средние коэфициенты за каждый год
yearly_coef1 = df_results['Коэфициент 1'].mean()
yearly_coef2 = df_results['Коэфициент 2'].mean()
print(f'\nСредние коэфициенты пролонгации за год: Коэфициент 1: {yearly_coef1}, Коэфициент 2: {yearly_coef2}')

#Агрегация по менеджерам
manager_results = []
for month in months:
  # вычисляем коэффициенты для каждого месяца
  coef1, coef2 = calculate_coef(df_merge, month)

  # добавляем информацию о коэффициентах в df_merge
  df_merge['Коэффициент 1'] = coef1
  df_merge['Коэффициент 2'] = coef2


#Группировка по менеджерам и вычисление средних коэфициентов
df_manager_grouped = df_merge.groupby('Account')[['Коэффициент 1', 'Коэффициент 2']].mean()
print(f'\nКоэфициенты пролонгации по менеджерам\n', df_manager_grouped)

#Средние значения за каждый месяц
monthly_coef1 = df_results['Коэффициент 1'].mean()
monthly_coef2 = df_results['Коэффициент 2'].mean()
print(f'\nСредние коэффициенты по отделу за месяц: Коэффициент 1: {monthly_coef1:.2f}, Коэффициент 2: {monthly_coef2:.2f}')



KeyError: 'December 2022'

In [21]:
df_merge

,id,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account,month,AM
0,42,36220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
1,657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
2,657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
3,594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
4,665,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,281,21450.0,13300.0,15900.0,19850.0,17350.0,14650.0,15900.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Соколова Анастасия Викторовна,декабрь 2023,Соколова Анастасия Викторовна
722,785,0.0,0.0,0.0,5306.6,12898.1,5287.0,10180.0,8600.0,3860.0,8600.0,700.0,700.0,0.0,0.0,0.0,0.0,Соколова Анастасия Викторовна,февраль 2023,Михайлов Андрей Сергеевич
723,785,0.0,0.0,0.0,5306.6,12898.1,5287.0,10180.0,8600.0,3860.0,8600.0,700.0,700.0,0.0,0.0,0.0,0.0,Соколова Анастасия Викторовна,декабрь 2023,Соколова Анастасия Викторовна
724,913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58200.0,58200.0,58200.0,58200.0,58200.0,58200.0,0.0,0.0,Смирнова Ольга Владимировна,декабрь 2023,Смирнова Ольга Владимировна


In [25]:
df_merge['Ноябрь 2022'].nunique()

141